In [32]:
import polars as pl

In [33]:
df_preppet = pl.read_parquet('kjoretoyinfo_preppet.parquet')

In [34]:
df_preppet

tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_aksler_drift,tekn_merke,tekn_modell,tekn_drivstoff,tekn_neste_pkk,er_elektrisk,tekn_farge,tekn_kjtgrp
date,date,i64,i64,str,str,date,bool,str,i64
2021-10-13,2021-10-13,2,3900,"""VITO""","""2""",2025-10-13,false,"""05""",315
2020-01-29,2020-01-29,2,5750,"""CADDY""","""2""",2026-01-29,false,"""06""",315
2023-11-29,2023-11-29,1,2600,"""Mustang Mach-E…","""5""",2027-11-29,true,"""06""",101
2020-01-27,2020-01-27,2,5750,"""CADDY""","""2""",2024-01-27,false,"""01""",315
2019-01-30,2019-01-30,2,2600,"""F-150 Platinum…","""2""",2024-05-03,false,"""06""",315
…,…,…,…,…,…,…,…,…,…
2003-11-27,2017-06-10,0,4350,"""ZAFIRA-A""","""1""",2021-04-30,false,"""11""",101
2014-01-17,2018-07-31,2,2600,"""RANGER""","""2""",2024-06-30,false,"""05""",315
2008-03-31,2023-07-28,0,3930,"""MX-5""","""1""",2025-08-16,false,"""05""",101


In [39]:
#11. Hvor stor andel av personbilene bilene som selges i Norge har firehjulstrekk?
firehjulstrekk_biler = df_preppet.filter(pl.col("tekn_aksler_drift") == 2)

andel_biler_solgt_firehjulstrekk = (firehjulstrekk_biler.shape[0] / df_preppet.shape[0] * 100)

print(f"Andelen av alle biler solgt med firehjulstrekk er: {andel_biler_solgt_firehjulstrekk}%")

Andelen av alle biler solgt med firehjulstrekk er: 33.22689353358721%


In [35]:
#12. Hvilken måned i året førstegangsregistreres det flest biler i Norge?
førstegangsregistreres_biler = df_preppet.filter(pl.col("tekn_reg_f_g_n").is_not_null())

mest_populær_måned = (
    førstegangsregistreres_biler.group_by(førstegangsregistreres_biler['tekn_reg_f_g_n'].dt.month())
    .agg(pl.col("tekn_reg_f_g_n").count().alias("count"))
    .sort("count", descending=True)
    .head(1)
)

print(f"Måneden med flest førstegangsregistreringer er: {mest_populær_måned}")


Måneden med flest førstegangsregistreringer er: shape: (1, 2)
┌────────────────┬────────┐
│ tekn_reg_f_g_n ┆ count  │
│ ---            ┆ ---    │
│ i8             ┆ u32    │
╞════════════════╪════════╡
│ 6              ┆ 248455 │
└────────────────┴────────┘


In [40]:
#14. Hvilken farge var den mest populære for traktorer i 2022? (se teknisk-kode.csv)
df_preppet = pl.read_parquet("kjoretoyinfo_preppet.parquet")

traktorer_2022 = df_preppet.filter(
    (pl.col("tekn_kjtgrp") == 401) &  
    (pl.col("tekn_reg_f_g_n").dt.year() == 2022)
)

populær_farge = traktorer_2022.group_by("tekn_farge").agg(pl.col("tekn_farge").count().alias("count"))

populær_farge = populær_farge.sort("count", descending=True)

populær_farge = populær_farge.get_column('tekn_farge')[0]

print(f"Den mest populære fargen for traktorer i 2022 var: {populær_farge}")


Den mest populære fargen for traktorer i 2022 var: 04
